In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
def converter(array):
    output = np.zeros_like(array)
    output[:, 0] = array[:, 1] * np.cos(np.radians(array[:, 0])) #real
    output[:, 1] = array[:, 1] * np.sin(np.radians(array[:, 0])) #im
    return output

In [4]:
path = Path(Path.cwd())

xls = pd.read_excel('Hospital_da_Luz_Base_de_dados_Lenstar_Cassini.xls', sheet_name=None)
cassini = xls['Cassini_full']
lenstar = xls['Lenstar']

#Tratamento de dados Cassini
""" 
    Remover Quality Factor Posterior Value inferior a 80
    Remover olhos repetidos (NHC e olho igual), mantendo o de maior QFPV
    Remover Pseudophakic
    Criar coluna com regra de acordo com axis
"""

filtered_cassini = cassini[cassini['Quality_Factor_Posterior_Value'] >= 80]
filtered_cassini = filtered_cassini.sort_values(by=['Quality_Factor_Posterior_Value'], ascending=[True])
filtered_cassini = filtered_cassini.drop_duplicates(subset=['NHC', 'C_Eye'], keep='last')
filtered_cassini = filtered_cassini.sort_values(by=['id'], ascending=[True])
filtered_cassini = filtered_cassini[filtered_cassini['C_Mode'] != 'Pseudophakic']

def categorize_value(x):
    if x <= 30 or x >= 150 :
        return 'ATR'
    elif x >= 60 and x <= 120:
        return 'WTR'
    else:
        return 'OBL'
    
filtered_cassini['C_TCA_Axis_360'] = filtered_cassini['C_TCA_Axis']*2
filtered_cassini['C_Ant_Axis_360'] = filtered_cassini['C_Ant_Axis']*2
filtered_cassini['C_Post_Axis_360'] = filtered_cassini['C_Post_Axis']*2

filtered_cassini['category_TCA'] = filtered_cassini['C_TCA_Axis'].apply(categorize_value)
filtered_cassini['category_Ant'] = filtered_cassini['C_Ant_Axis'].apply(categorize_value)
filtered_cassini['category_Post'] = filtered_cassini['C_Post_Axis'].apply(categorize_value)

filtered_cassini['C_TCA_parte_real'] = converter(np.column_stack((filtered_cassini['C_TCA_Axis_360'], filtered_cassini['C_TCA_Ast'])))[:,0]
filtered_cassini['C_TCA_parte_imaginaria'] = converter(np.column_stack((filtered_cassini['C_TCA_Axis_360'], filtered_cassini['C_TCA_Ast'])))[:,1]
filtered_cassini['C_Ant_parte_real'] = converter(np.column_stack((filtered_cassini['C_Ant_Axis_360'], filtered_cassini['C_Ant_Ast'])))[:,0]
filtered_cassini['C_Ant_parte_imaginaria'] = converter(np.column_stack((filtered_cassini['C_Ant_Axis_360'], filtered_cassini['C_Ant_Ast'])))[:,1]
filtered_cassini['C_Post_parte_real'] = converter(np.column_stack((filtered_cassini['C_Post_Axis_360'], filtered_cassini['C_Post_Ast'])))[:,0]
filtered_cassini['C_Post_parte_imaginaria'] = converter(np.column_stack((filtered_cassini['C_Post_Axis_360'], filtered_cassini['C_Post_Ast'])))[:,1]

geral_df = filtered_cassini
geral_df.to_excel( path / 'ficheiros' / 'cassini_geral.xlsx')

atr_df = geral_df[geral_df['category_Ant'] == 'ATR']
atr_df.to_excel( path / 'ficheiros' / 'cassini_atr.xlsx')

wtr_df = geral_df[geral_df['category_Ant'] == 'WTR']
wtr_df.to_excel( path / 'ficheiros' / 'cassini_wtr.xlsx')

obl_df = geral_df[geral_df['category_Ant'] == 'OBL']
obl_df.to_excel( path / 'ficheiros' / 'cassini_obl.xlsx')

TimeoutError: [Errno 60] Operation timed out: '/Users/constancacachim/Library/CloudStorage/OneDrive-UniversidadedeLisboa/IST/GOLP/codigo/ficheiros/cassini_geral.xlsx'

In [ ]:
# #Tratamento de dados Lenstar

# """ 
#     Remover olhos repetidos (NHC e olho igual), escolha arbitrária
#     Remover Pseudophakic e Aphakic
# """

# filtered_lenstar = lenstar[lenstar['L_Mode'] != 'Pseudophakic']
# filtered_lenstar = filtered_lenstar[filtered_lenstar['L_Mode'] != 'Aphakic']
# filtered_lenstar = filtered_lenstar.drop_duplicates(subset=['NHC', 'L_Eye'], keep='last')

# filtered_lenstar.to_excel('/Users/constancacachim/Desktop/golp_dados/ficheiros/filtered_lenstar.xlsx', index=False, sheet_name='lenstar')

# """
#     Merge cassini e lenstar com base em NHC
# """
# cassini_lenstar = pd.merge(filtered_lenstar,filtered_cassini, on='NHC', how='outer')
# cassini_lenstar.to_excel('/Users/constancacachim/Desktop/golp_dados/ficheiros/filtered_cassini_lenstar.xlsx', index=False, sheet_name='merge')